### UCB–USACE LSTMs — Daily Grid‑Search + Best Model Evaluation (Tule)

In [1]:
BASIN = "tule"  # No need to touch this
MODE = "daily" # No need to touch this

### 0) Quick switches (edit these)

##### - `run_gridsearch`: True - perform a full grid‑search now. False - reuse the last saved best params.
##### - `skip_best_model_validation` / `skip_best_model_test`:
#####       True: reuse previously saved artifacts (requires `READ_STAMP` to match an archived run).
#####       False: re‑train the best models and regenerate metrics/plots now.
##### - `GPU_SETTING`:
#####       0: GPU - CUDA: 0 (if available)
#####      -1: default CPU

##### - `RUN_LABEL`: ties together series of runs (e.g., "BASELINE", "EXPERIMENT_X"). If you change this, you will get a new shared folder and separate hyperparams CSV.
##### - `READ_STAMP`: when *skipping* re‑runs, tells the notebook which archived stamp to read from. This is to ensure you can reproduce exact results later. ONE MORE SENTENCE WITH EXAMPLE FILEPATH

In [2]:
run_gridsearch = False  
skip_best_model_validation = True  
skip_best_model_test = True 
verbose = False

GPU_SETTING = -1 
NUM_ENSEMBLES = 5 

RUN_LABEL = "BASELINE" # experiment tag
READ_STAMP = "20250815T000000Z" # read artifacts stamp when skipping re-runs

DATA_DIR = "/Desktop/UCB-USACE-RR-PROJECT/tule_river_data"

In [3]:
import sys
from pathlib import Path

def _find_repo_root(start=None) -> Path:
    """Walk up from `start` until we see the repo markers."""
    here = Path(start or Path.cwd()).resolve()
    for p in [here, *here.parents]:
        if (p / "UCB_training").exists() and (p / "neuralhydrology").exists():
            return p
        if p.name == "UCB-USACE-RR-PROJECT":
            return p
    raise FileNotFoundError("Could not locate the repo root. Open the notebook from inside the repo.")

REPO_ROOT = _find_repo_root()
if str(REPO_ROOT) not in sys.path:
    sys.path.insert(0, str(REPO_ROOT))
print("[path] repo root =", REPO_ROOT)

# Now imports will work:
from UCB_training.UCB_utils import repo_root

# Point to tule_river_data robustly and sanity‑check required files
DATA_DIR = (repo_root() / "tule_river_data").resolve()
print("DATA_DIR =", DATA_DIR)

for fn in ("daily_shift.csv", "Tule_daily_shift.csv"):
    p = DATA_DIR / fn
    print("exists:", p, p.exists())
    assert p.exists(), f"Missing required file: {p}"

[path] repo root = /Users/canruso/Desktop/UCB-USACE-RR-PROJECT
DATA_DIR = /Users/canruso/Desktop/UCB-USACE-RR-PROJECT/tule_river_data
exists: /Users/canruso/Desktop/UCB-USACE-RR-PROJECT/tule_river_data/daily_shift.csv True
exists: /Users/canruso/Desktop/UCB-USACE-RR-PROJECT/tule_river_data/Tule_daily_shift.csv True


In [4]:
from pathlib import Path
from UCB_training.UCB_utils import repo_root

# Robust: point to the repo’s tule_river_data no matter where the notebook runs
DATA_DIR = str((repo_root() / "tule_river_data").resolve())
print("DATA_DIR =", DATA_DIR)

# Sanity check the two files Tuler needs
for fn in ("daily_shift.csv", "Tule_daily_shift.csv"):
    p = Path(DATA_DIR) / fn
    print(p, "exists?" , p.exists())
    assert p.exists(), f"Missing required file: {p}"

DATA_DIR = /Users/canruso/Desktop/UCB-USACE-RR-PROJECT/tule_river_data
/Users/canruso/Desktop/UCB-USACE-RR-PROJECT/tule_river_data/daily_shift.csv exists? True
/Users/canruso/Desktop/UCB-USACE-RR-PROJECT/tule_river_data/Tule_daily_shift.csv exists? True


#### 1) Hyperparameter space

##### - `schedule_pairs` has two tuples: (fractions, learning_rates) that is converted to a piecewise LR schedule;
##### Ex: over 48 epochs, the input ((0.5, 0.25), (0.01, 0.005, 0.001)) will lead to → 0–24: 0.01, 24–36: 0.005, 36–48: 0.001 

##### Everything else is model architectucture/training related. The notebook is equipped to handle other hyperparameters, but they have to be part of the NH library as the Config object expects them.

##### Remember that there will be `len(hyperparam_space[hp1]) * len(hyperparam_space[hp2]) * ...` total runs. So gridsearch can be computationally expensive.

In [5]:
hyperparam_space = {
    "hidden_size": [64, 128, 256],
    "output_dropout": [0.4],
    "seq_length": [90, 120],
    "num_layers": [1],
    "epochs": [16, 32, 48],
    "batch_size": [64, 128, 256],
    "schedule_pairs": [
        ((0.5, 0.25), (0.01, 0.005, 0.001))
    ]
}

#### 2) Imports and setup

In [6]:
import sys
import pandas as pd
import os
import itertools
from pathlib import Path
from tqdm import tqdm
import warnings
from datetime import datetime
warnings.simplefilter(action='ignore', category=FutureWarning)

In [7]:
current_dir = os.getcwd()
print(current_dir)

/Users/canruso/Desktop/UCB-USACE-RR-PROJECT/notebooks/basins/tule


In [8]:
# Add the repository root to sys.path (without hard-coding usernames/drives).

library_path = os.path.join('..', '..', '..','..','UCB-USACE-RR-PROJECT')
sys.path.insert(0, library_path)
print(sys.path)

['../../../../UCB-USACE-RR-PROJECT', '/Users/canruso/Desktop/UCB-USACE-RR-PROJECT', '/Users/canruso/Desktop/UCB-USACE-RR-PROJECT/notebooks/basins/tule', '/Users/canruso/Desktop', '/Users/canruso/miniforge3/envs/ESDL_LSTM/lib/python310.zip', '/Users/canruso/miniforge3/envs/ESDL_LSTM/lib/python3.10', '/Users/canruso/miniforge3/envs/ESDL_LSTM/lib/python3.10/lib-dynload', '', '/Users/canruso/miniforge3/envs/ESDL_LSTM/lib/python3.10/site-packages', '/Users/canruso/miniforge3/envs/ESDL_LSTM/lib/python3.10/site-packages/setuptools/_vendor']


In [9]:
from neuralhydrology.evaluation.metrics import *
from UCB_training.UCB_train import UCB_trainer
from UCB_training.UCB_utils import (fractional_multi_lr, write_paths, to_path_or_list, ensure_output_tree, set_active_context, data_dir, repo_root, get_output_dir, make_run_stamp, get_yaml_path, ctx_for, hparams_exists, save_hparams, load_hparams, runs_latest_path, archive_runs_json, read_csv_artifact, ensure_shared_tree)
from UCB_training.UCB_plotting import (plot_timeseries_comparison, scatter_triptych_pngs_v3, ts_triptych_v3)

In [10]:
current_path = os.getcwd()
library_path = current_path.split('UCB-USACE-RR-PROJECT')[0] + 'UCB-USACE-RR-PROJECT'

##### 3) Stamping & run registry

##### - `RUNS_FILE`: a **LATEST** JSON file that stores the filepath for this basin/mode/label.
##### - `RUN_STAMP`: unique UTC stamp if we plan to (re)run models. It will correspond to the current UTC time when the cell is run. 
##### - `ACTIVE_STAMP`: stamp we will **write under** (if re-running) or **read from** (if skipping).

##### **Rule of thumb**
##### - If you’re skipping validation/test re-runs, set `READ_STAMP` to an archived stamp that already contains the needed artifacts.

In [11]:
RUNS_FILE = str(runs_latest_path(BASIN, MODE, RUN_LABEL)) 
SHOULD_STAMP = not (run_gridsearch or (skip_best_model_validation and skip_best_model_test))
RUN_STAMP = make_run_stamp() if SHOULD_STAMP else None
ACTIVE_STAMP = RUN_STAMP if RUN_STAMP is not None else READ_STAMP

#### 4) Activate context + ensure output tree

##### This sets up the active context used by our path router so that outputs (plots/metrics/CSVs) go into:  
##### `outputs/<BASIN>/<MODE>[/<RUN_LABEL>_<RUN_STAMP>]/{timeseries,metrics,plots,...}`

In [12]:
set_active_context(basin=BASIN, resolution=MODE, run_stamp=ACTIVE_STAMP, run_tag=RUN_LABEL, append_stamp_to_filenames=False)
SHARED_FOLDER = ensure_shared_tree(BASIN, MODE)
RUNS_PARENT = SHARED_FOLDER / "runs" / (f"{RUN_LABEL}_{RUN_STAMP}" if RUN_STAMP else RUN_LABEL)

print("NH runs will be written under:")
print(RUNS_PARENT.resolve())

NH runs will be written under:
/Users/canruso/Desktop/UCB-USACE-RR-PROJECT/outputs/tule/daily_shared/runs/BASELINE


#### 5) Data & feature definitions

##### - `path_to_csv`: repository data folder (daily CSVs for Tule, etc.). The data_dir() utility finds UCB-USACE-LSTMs/russian_river_data
##### - `path_to_yaml`: base NH config (we override several inputs for this config from the notebook).
##### - `path_to_physics_data`: Where the HMS data is

In [13]:
path_to_csv = Path(DATA_DIR)
path_to_yaml = get_yaml_path("tule")
path_to_physics_data = path_to_csv / "Tule_daily_shift.csv"

#### 6) Physics‑informed feature list

##### These are **dynamic_inputs** passed when `physics_informed=True`. They augment the LSTM model with additional HMS‑derived features.
##### If you want to remove some features from the PILSTM, remove them from this list. If you want to remove features from the LSTM, remove them from the yaml file.

In [14]:
features_with_physics = [
    'TuleR_S10_PRECIP_INC_IN',
    'TuleR_S10_SWE_OBSERVED_IN',
    'TuleR_S10_TEMPERATURE_AIR_DEGF',
    'MF_TuleR_S10_ET_POTENTIAL_IN',
    'MF_TuleR_S10_FLOW_CFS',
    'MF_TuleR_S10_SATURATION_FRACTION_ININ',
    'MF_TuleR_S10_STORAGE_GW_1_IN',
    'MF_TuleR_S10_STORAGE_GW_2_IN',
    'MF_TuleR_S10_STORAGE_SOIL_IN',
    'MF_TuleR_S20_ET_POTENTIAL_IN',
    'MF_TuleR_S20_FLOW_CFS',
    'MF_TuleR_S20_SATURATION_FRACTION_ININ',
    'MF_TuleR_S20_STORAGE_GW_1_IN',
    'MF_TuleR_S20_STORAGE_GW_2_IN',
    'MF_TuleR_S20_STORAGE_SOIL_IN',
    'NF_TuleR_S10_ET_POTENTIAL_IN',
    'NF_TuleR_S10_FLOW_CFS',
    'NF_TuleR_S10_SATURATION_FRACTION_ININ',
    'NF_TuleR_S10_STORAGE_GW_1_IN',
    'NF_TuleR_S10_STORAGE_GW_2_IN',
    'NF_TuleR_S10_STORAGE_SOIL_IN',
    'SF_TuleR_S10_ET_POTENTIAL_IN',
    'SF_TuleR_S10_FLOW_CFS',
    'SF_TuleR_S10_SATURATION_FRACTION_ININ',
    'SF_TuleR_S10_STORAGE_GW_1_IN',
    'SF_TuleR_S10_STORAGE_GW_2_IN',
    'SF_TuleR_S10_STORAGE_SOIL_IN',
    'TuleR_S10_ET_POTENTIAL_IN',
    'TuleR_S10_FLOW_CFS',
    'TuleR_S10_SATURATION_FRACTION_ININ',
    'TuleR_S10_STORAGE_GW_1_IN',
    'TuleR_S10_STORAGE_GW_2_IN',
    'TuleR_S10_STORAGE_SOIL_IN',
    'TuleR_S20_ET_POTENTIAL_IN',
    'TuleR_S20_FLOW_CFS',
    'TuleR_S20_SATURATION_FRACTION_ININ',
    'TuleR_S20_STORAGE_GW_1_IN',
    'TuleR_S20_STORAGE_GW_2_IN',
    'TuleR_S20_STORAGE_SOIL_IN'
]


In [15]:
no_physics_results = []
physics_results = []

In [16]:
start_time = datetime.utcnow()
print("Start time:", start_time.strftime("%Y-%m-%d %H:%M:%S"))

Start time: 2025-10-23 04:29:46


#### 7) Grid‑search: run or reuse

##### - If `run_gridsearch=True` (or no best‑params cached yet), we iterate all combinations, train, evaluate on **validation**, and collect metrics into the following dataframes:
#####   `*_no_physics_gridsearch.csv` and `*_physics_gridsearch.csv`.
##### - We then save a **best‑params CSV** plus the full grid results into:
#####   `outputs/<basin>/<mode>_shared/hyperparams/`.
##### If `run_gridsearch=False` and best params CSV exists, we **skip** to loading them.

In [17]:
hyperparam_names = []
for i, hp in enumerate(hyperparam_space.keys()):
    hyperparam_names.append(hp)

total_iters = 1
for name in hyperparam_names:
    total_iters *= len(hyperparam_space[name])

if run_gridsearch or not hparams_exists(BASIN, MODE, RUN_LABEL):
    for combinations in tqdm(
        itertools.product(*[hyperparam_space[hp] for hp in hyperparam_names]), 
        desc="Grid-Search", 
        total=total_iters, 
        unit="it", 
        ncols=60, 
        ascii=True):

        hp_run = {}
        j = 0
        schedule_pairs = None
        
        while j < len(hyperparam_names):
            name = hyperparam_names[j]
            val = combinations[j]

            if name == "seq_length":
                hp_run["seq_length"] = val

            elif name == "schedule_pairs":
                schedule_pairs = val
                
            else:
                hp_run[name] = val

            j += 1

        if schedule_pairs is not None:
            fractions, rates = schedule_pairs
            hp_run["learning_rate"] = fractional_multi_lr(
                epochs=int(hp_run["epochs"]), fractions=list(fractions), lrs=list(rates))
        else:
            hp_run.setdefault("learning_rate", {0: 0.01, 30: 0.005, 40: 0.001})

        if verbose:
            print(f"\nNoPhysics: {hp_run}")
            
        trainer = UCB_trainer(
            path_to_csv_folder=path_to_csv,
            yaml_path=path_to_yaml,
            hyperparams=hp_run,
            input_features=None,
            physics_informed=False,
            physics_data_file=None,
            hourly=False,  
            extend_train_period=False,
            gpu=GPU_SETTING,
            verbose=verbose,
            runs_parent=RUNS_PARENT,
            run_label=RUN_LABEL,
            run_stamp=RUN_STAMP)
        
        trainer.train()
        csv_path, metrics_dict = trainer.results()
        
        row_data = {}
        j = 0
        while j < len(hyperparam_names):
            row_data[hyperparam_names[j]] = combinations[j]
            j += 1

        row_data["learning_rate"] = str(hp_run["learning_rate"])
            
        for k, v in metrics_dict.items():
            row_data[k] = v
        
        no_physics_results.append(row_data)

    df_no_physics = pd.DataFrame(no_physics_results)
    df_no_physics.sort_values(by="NSE", ascending=False, inplace=True)
    df_no_physics.reset_index(drop=True, inplace=True)
else:
    print("Skipping grid search!")

Grid-Search:  96%|#####7| 52/54 [12:24:14<28:37, 858.75s/it]

KeyboardInterrupt



In [ ]:
if run_gridsearch or not hparams_exists(BASIN, MODE, RUN_LABEL):   
    for combinations in tqdm(
        itertools.product(*[hyperparam_space[hp] for hp in hyperparam_names]), 
        desc="Grid-Search", 
        total=total_iters, 
        unit="it", 
        ncols=60, 
        ascii=True):

        hp_run = {}
        j = 0
        schedule_pairs = None
        
        while j < len(hyperparam_names):
            name = hyperparam_names[j]
            val  = combinations[j]

            if name == "seq_length":
                hp_run["seq_length"] = val

            elif name == "schedule_pairs":
                schedule_pairs = val

            else:
                hp_run[name] = val

            j += 1

        if schedule_pairs is not None:
            fractions, rates = schedule_pairs
            hp_run["learning_rate"] = fractional_multi_lr(
                epochs=int(hp_run["epochs"]), fractions=list(fractions), lrs=list(rates))
        else:
            hp_run.setdefault("learning_rate", {0: 0.01, 30: 0.005, 40: 0.001})
            
        if verbose:
            print(f"\nPhysics: {hp_run}")
            
        trainer = UCB_trainer(
            path_to_csv_folder=path_to_csv,
            yaml_path=path_to_yaml,
            hyperparams=hp_run,
            input_features=features_with_physics,
            physics_informed=True,
            physics_data_file=path_to_physics_data,
            hourly=False,
            extend_train_period=False,
            gpu=GPU_SETTING,
            verbose=verbose,
            runs_parent=RUNS_PARENT,
            run_label=RUN_LABEL,
            run_stamp=RUN_STAMP)
        
        trainer.train()
        csv_path, metrics_dict = trainer.results()

        row_data = {}
        j = 0
        while j < len(hyperparam_names):
            row_data[hyperparam_names[j]] = combinations[j]
            j += 1

        row_data["learning_rate"] = str(hp_run["learning_rate"])

        for k, v in metrics_dict.items():
            row_data[k] = v
            
        physics_results.append(row_data)
    df_physics = pd.DataFrame(physics_results)
    df_physics.sort_values(by="NSE", ascending=False, inplace=True)
    df_physics.reset_index(drop=True, inplace=True)
    best_no_phys = df_no_physics.iloc[0].to_dict()
    best_phys = df_physics.iloc[0].to_dict()
    best_no_phys["model_type"] = "no_physics"
    best_phys["model_type"] = "physics"
    best_params_df = pd.DataFrame([best_no_phys, best_phys])
    save_hparams(best_df=best_params_df, basin=BASIN, mode=MODE, label=RUN_LABEL, run_stamp=RUN_STAMP, df_no=df_no_physics, df_phys=df_physics)
else:
    print("Skipping grid search!")

#### 8) Load best hyperparameters (if not just saved)

##### The file containing each hyperparameter combination's performance lives in:
##### `outputs/<basin>/<mode>_shared/hyperparams/<basin>_<mode>_<RUN_LABEL>_hyperparams.csv`

In [ ]:
try:
    if run_gridsearch:
        print("\n[INFO] Using best_params_df from the just-completed grid search (ignoring READ_STAMP).")
    else:
        print("\nLoading best hyperparams from CSV...")
        best_params_df = load_hparams(BASIN, MODE, RUN_LABEL, stamp=READ_STAMP)
except FileNotFoundError as e:
    raise SystemExit(f"[ERROR] {e}  (Set run_gridsearch=True to generate it.)")

best_no_phys = best_params_df.query("model_type == 'no_physics'").iloc[0].to_dict()
best_phys = best_params_df.query("model_type == 'physics'").iloc[0].to_dict()

best_no_physics_params = {}
j = 0
while j < len(hyperparam_names):
    name = hyperparam_names[j]
    if name == "output_dropout":
        best_no_physics_params[name] = float(best_no_phys[name])
        j += 1

    elif name == "seq_length":
        best_no_physics_params["seq_length"] = int(best_no_phys["seq_length"])
        j += 1

    elif name == "schedule_pairs":
        j += 1

    else:
        best_no_physics_params[name] = int(best_no_phys[name])
        j += 1

if "learning_rate" in best_no_phys and pd.notna(best_no_phys["learning_rate"]):
    best_no_physics_params["learning_rate"] = eval(str(best_no_phys["learning_rate"]))

elif "schedule_pairs" in best_no_phys and pd.notna(best_no_phys["schedule_pairs"]):
    sp = best_no_phys["schedule_pairs"]
    if isinstance(sp, str):
        sp = eval(sp)
    fractions, rates = sp
    best_no_physics_params["learning_rate"] = fractional_multi_lr(
        epochs=int(best_no_physics_params["epochs"]),
        fractions=list(fractions),
        lrs=list(rates))

else:
    best_no_physics_params["learning_rate"] = {0: 0.01, 30: 0.005, 40: 0.001}

best_physics_params = {}
j = 0
while j < len(hyperparam_names):
    name = hyperparam_names[j]
    if name == "output_dropout":
        best_physics_params[name] = float(best_phys[name])
        j += 1

    elif name == "seq_length":
        best_physics_params["seq_length"] = int(best_phys["seq_length"])
        j += 1

    elif name == "schedule_pairs":
        j += 1

    else:
        best_physics_params[name] = int(best_phys[name])
        j += 1

if "learning_rate" in best_phys and pd.notna(best_phys["learning_rate"]):
    best_physics_params["learning_rate"] = eval(str(best_phys["learning_rate"]))

elif "schedule_pairs" in best_phys and pd.notna(best_phys["schedule_pairs"]):
    sp = best_phys["schedule_pairs"]
    if isinstance(sp, str):
        sp = eval(sp)
    fractions, rates = sp
    best_physics_params["learning_rate"] = fractional_multi_lr(
        epochs=int(best_physics_params["epochs"]),
        fractions=list(fractions),
        lrs=list(rates))

else:
    best_physics_params["learning_rate"] = {0: 0.01, 30: 0.005, 40: 0.001}

print("Loaded best hyperparams from CSV:")
print("Best NO-PHYS:", best_no_physics_params)
print("Best PHYS:", best_physics_params)

#### 9a) Re-run validation period with best hyperparameters

In [ ]:
if not skip_best_model_validation:
    lstmNoPhysicsValBest = UCB_trainer(
        path_to_csv_folder=path_to_csv,
        yaml_path=path_to_yaml,
        hyperparams=best_no_physics_params,
        input_features=None,
        physics_informed=False,
        physics_data_file=None,
        hourly=False,
        extend_train_period=False,
        gpu=GPU_SETTING,
        num_ensemble_members = NUM_ENSEMBLES,
        verbose=verbose,
        runs_parent=RUNS_PARENT,
        run_label=RUN_LABEL,
        run_stamp=RUN_STAMP)
    
    lstmNoPhysicsValBest.train()
    no_physics_val_csv, no_physics_val_metrics = lstmNoPhysicsValBest.results()
    no_physics_val_metrics

In [ ]:
if not skip_best_model_validation:
    lstmPhysicsValBest = UCB_trainer(
        path_to_csv_folder=path_to_csv,
        yaml_path=path_to_yaml,
        hyperparams=best_physics_params,
        input_features=features_with_physics,
        physics_informed=True,
        physics_data_file=path_to_physics_data,
        hourly=False,
        extend_train_period=False,
        gpu=GPU_SETTING,
        num_ensemble_members = NUM_ENSEMBLES,
        verbose=verbose,
        runs_parent=RUNS_PARENT,
        run_label=RUN_LABEL,
        run_stamp=RUN_STAMP)
    
    lstmPhysicsValBest.train()
    physics_val_csv, physics_val_metrics = lstmPhysicsValBest.results()
    physics_val_metrics

In [ ]:
if not skip_best_model_validation:
    plot_timeseries_comparison(source=(no_physics_val_csv, physics_val_csv, path_to_physics_data), title="Tule Basin Daily Model Comparison (Validation)", backend="mpl", metrics=["NSE", "PBIAS"], metrics_out="tule_daily_val_metrics.csv", ts_out="tule_daily_val_combined_ts.csv", fig_out="tule_daily_val_model_comparison.png", legend_font=20, axis_font=22)
else:
    combined_df_val = read_csv_artifact("tule_daily_val_combined_ts.csv", kind="csv", period="validation", stamp = READ_STAMP, run_label = RUN_LABEL)
    plot_timeseries_comparison(source=combined_df_val, title="Tule Basin Daily Model Comparison (Validation)", backend="mpl", metrics=["NSE", "PBIAS"], metrics_out="tule_daily_val_metrics.csv", ts_out="tule_daily_val_combined_ts.csv", fig_out="tule_daily_val_model_comparison.png", legend_font=20, axis_font=22)

In [ ]:
if skip_best_model_validation:
    val_metrics = read_csv_artifact("tule_daily_val_metrics.csv", kind="metrics", period="validation", index_col=0, stamp = READ_STAMP, run_label = RUN_LABEL)
    print(val_metrics)

#### 9b) Re-run test period with best hyperparameters

In [ ]:
if not skip_best_model_test:
    lstmNoPhysicsExtBest = UCB_trainer(
        path_to_csv_folder=path_to_csv,
        yaml_path=path_to_yaml,
        hyperparams=best_no_physics_params,
        input_features=None,
        physics_informed=False,
        physics_data_file=None,
        hourly=False,
        extend_train_period=True,  
        gpu=GPU_SETTING,
        num_ensemble_members = NUM_ENSEMBLES,
        verbose=verbose,
        runs_parent=RUNS_PARENT,
        run_label=RUN_LABEL,
        run_stamp=RUN_STAMP)
    
    lstmNoPhysicsExtBest.train()
    no_physics_test_csv, no_physics_test_metrics = lstmNoPhysicsExtBest.results('test')
    no_physics_test_metrics

In [ ]:
if not skip_best_model_test:
    lstmPhysicsExtBest = UCB_trainer(
        path_to_csv_folder=path_to_csv,
        yaml_path=path_to_yaml,
        hyperparams=best_physics_params,
        input_features=features_with_physics,
        physics_informed=True,
        physics_data_file=path_to_physics_data,
        hourly=False,
        extend_train_period=True,
        gpu=GPU_SETTING,
        num_ensemble_members = NUM_ENSEMBLES,
        verbose=verbose,
        runs_parent=RUNS_PARENT,
        run_label=RUN_LABEL,
        run_stamp=RUN_STAMP)
    
    lstmPhysicsExtBest.train()
    physics_test_csv, physics_test_metrics = lstmPhysicsExtBest.results('test')
    physics_test_metrics

In [ ]:
if not skip_best_model_test:
    plot_timeseries_comparison(source=(no_physics_test_csv, physics_test_csv, path_to_physics_data), title="Tule Basin Daily Model Comparison (Test)", backend="mpl", metrics=["NSE", "PBIAS"], metrics_out="tule_daily_test_metrics.csv", ts_out="tule_daily_test_combined_ts.csv", fig_out="tule_daily_test_model_comparison.png", legend_font=20, axis_font=22)
else:
    combined_df = read_csv_artifact("tule_daily_test_combined_ts.csv", kind="csv", period="test", stamp = READ_STAMP, run_label = RUN_LABEL)
    plot_timeseries_comparison(source=combined_df, title="Tule Basin Daily Model Comparison (Test)", backend="mpl", metrics=["NSE", "PBIAS"], metrics_out="tule_daily_test_metrics.csv", ts_out="tule_daily_test_combined_ts.csv", fig_out="tule_daily_test_model_comparison.png", legend_font=20, axis_font=22)

In [ ]:
if not skip_best_model_test:
    plot_timeseries_comparison(source=(no_physics_test_csv, physics_test_csv, path_to_physics_data), title="Tule Basin Daily Model Comparison (Test)", backend="plotly", metrics=["NSE", "PBIAS"], metrics_out="tule_daily_test_metrics.csv", ts_out="tule_daily_test_combined_ts.csv", fig_out="tule_daily_test_model_comparison.png", legend_font=12, axis_font=22)
else:
    plot_timeseries_comparison(source=combined_df, title="Tule Basin Daily Model Comparison (Test)", backend="plotly", metrics=["NSE", "PBIAS"], metrics_out="tule_daily_test_metrics.csv", ts_out="tule_daily_test_combined_ts.csv", fig_out="tule_daily_test_model_comparison.png", legend_font=12, axis_font=22)

In [ ]:
test_metrics = read_csv_artifact("tule_daily_test_metrics.csv", kind="metrics", period="test", index_col=0, stamp = READ_STAMP, run_label = RUN_LABEL)
print(test_metrics)

In [ ]:
if not skip_best_model_test:
    write_paths("no_physics", lstmNoPhysicsExtBest, filename = RUNS_FILE)
    write_paths("physics", lstmPhysicsExtBest, filename = RUNS_FILE)
    archived_path = archive_runs_json(Path(RUNS_FILE), BASIN, MODE, RUN_LABEL, RUN_STAMP)

In [ ]:
end_time = datetime.utcnow()
print("\nEnd time:", end_time.strftime("%Y-%m-%d %H:%M:%S"))
print("Total time:", end_time - start_time)

#### 10) Additional plots (Validation/Test; whole, wettest, driest water‑years); timeseries + scatter triptychs
##### We reuse either the freshly generated combined timeseries or the archived CSVs (depending on skip flags).
##### Note: For more detail, all these functions are in **`UCB_training/UCB_plotting.py`**

In [ ]:
if skip_best_model_validation:
    combined_df_val = read_csv_artifact("tule_daily_val_combined_ts.csv", kind="csv", period="validation", stamp = READ_STAMP, run_label = RUN_LABEL)
if skip_best_model_test:
    combined_df = read_csv_artifact("tule_daily_test_combined_ts.csv", kind="csv", period="test", stamp = READ_STAMP, run_label = RUN_LABEL)

In [ ]:
metric_list = ["NSE", "PBIAS"]

wettest_start_val = "2003-10-01"
wettest_end_val = "2004-09-30"
dryest_start_val = "2004-10-01"
dryest_end_val = "2005-09-30"
wettest_start_test = "2005-10-01"
wettest_end_test = "2006-09-30"
dryest_start_test = "2008-10-01"
dryest_end_test = "2009-09-30"

In [ ]:
if not skip_best_model_validation:
    plot_timeseries_comparison(source=(no_physics_val_csv, physics_val_csv, path_to_physics_data), title="Tule Daily Validation Timeseries", backend="mpl", metrics=metric_list, metrics_out="tule_daily_val_metrics.csv", ts_out="tule_daily_val_combined_ts.csv", fig_out="tule_daily_val_model_comparison.png", legend_font=20, axis_font=22)
else:
    plot_timeseries_comparison(source=combined_df_val, title="Tule Daily Validation Timeseries", backend="mpl", metrics=metric_list, metrics_out="tule_daily_val_metrics.csv", ts_out="tule_daily_val_combined_ts.csv", fig_out="tule_daily_val_model_comparison.png", legend_font=20, axis_font=22)

In [ ]:
if not skip_best_model_test:
    plot_timeseries_comparison(source=(no_physics_test_csv, physics_test_csv, path_to_physics_data), title="Tule Daily Test Timeseries", backend="mpl", metrics=metric_list, metrics_out="tule_daily_test_metrics.csv", ts_out="tule_daily_test_combined_ts.csv", fig_out="tule_daily_test_model_comparison.png", legend_font=20, axis_font=22)
else:
    plot_timeseries_comparison(source=combined_df, title="Tule Daily Test Timeseries", backend="mpl", metrics=metric_list, metrics_out="tule_daily_test_metrics.csv", ts_out="tule_daily_test_combined_ts.csv", fig_out="tule_daily_test_model_comparison.png", legend_font=20, axis_font=22)

In [ ]:
if not skip_best_model_test:
    plot_timeseries_comparison(source=(no_physics_test_csv, physics_test_csv, path_to_physics_data), title="Tule Daily Test Timeseries – Interactive", backend="plotly", metrics=metric_list, metrics_out="tule_daily_test_metrics.csv", ts_out="tule_daily_test_combined_ts.csv", legend_font=12, axis_font=22)
else:
    plot_timeseries_comparison(source=combined_df, title="Tule Daily Timeseries – Interactive", backend="plotly", metrics=metric_list, metrics_out="tule_daily_test_metrics.csv", ts_out="tule_daily_test_combined_ts.csv", legend_font=12, axis_font=22)

### Wettest Year Performance

In [ ]:
if not skip_best_model_validation:
    plot_timeseries_comparison(source=(no_physics_val_csv, physics_val_csv, path_to_physics_data), title="Tule Daily Wettest Year Validation Timeseries", backend="mpl", metrics=metric_list, start_date=wettest_start_val, end_date=wettest_end_val, metrics_out="tule_daily_val_wet_metrics.csv", ts_out="tule_daily_val_wet_combined_ts.csv", fig_out="tule_daily_val_wet_model_comparison.png", legend_font=20, axis_font=22)
else:
    plot_timeseries_comparison(source=combined_df_val, title="Tule Daily Wettest Year Validation Timeseries", backend="mpl", metrics=metric_list, start_date=wettest_start_val, end_date=wettest_end_val, metrics_out="tule_daily_val_wet_metrics.csv", ts_out="tule_daily_val_wet_combined_ts.csv", fig_out="tule_daily_val_wet_model_comparison.png", legend_font=20, axis_font=22)

In [ ]:
if not skip_best_model_test:
    plot_timeseries_comparison(source=(no_physics_test_csv, physics_test_csv, path_to_physics_data), title="tule Daily Wettest Year Test Timeseries", backend="mpl", metrics=metric_list, start_date=wettest_start_test, end_date=wettest_end_test, metrics_out="tule_daily_test_wet_metrics.csv", ts_out="tule_daily_test_wet_combined_ts.csv", fig_out="tule_daily_test_wet_model_comparison.png", legend_font=20, axis_font=22)
else:
    plot_timeseries_comparison(source=combined_df, title="tule Daily Wettest Year Test Timeseries", backend="mpl", metrics=metric_list, start_date=wettest_start_test, end_date=wettest_end_test, metrics_out="tule_daily_test_wet_metrics.csv", ts_out="tule_daily_test_wet_combined_ts.csv", fig_out="tule_daily_test_wet_model_comparison.png", legend_font=20, axis_font=22)

In [ ]:
if not skip_best_model_test:
    plot_timeseries_comparison(source=(no_physics_test_csv, physics_test_csv, path_to_physics_data), title="Tule Daily Wettest Year Test Timeseries – Interactive", backend="plotly", metrics=metric_list, start_date=wettest_start_test, end_date=wettest_end_test, metrics_out="tule_daily_test_wet_metrics.csv", ts_out="tule_daily_test_wet_combined_ts.csv", legend_font=12, axis_font=22)
else:
    plot_timeseries_comparison(source=combined_df, title="Tule Daily Wettest Year Test Timeseries – Interactive", backend="plotly", metrics=metric_list, start_date=wettest_start_test, end_date=wettest_end_test, metrics_out="tule_daily_test_wet_metrics.csv", ts_out="tule_daily_test_wet_combined_ts.csv", legend_font=12, axis_font=22)

##### Dryest Year Performance

In [ ]:
if not skip_best_model_validation:
    plot_timeseries_comparison(source=(no_physics_val_csv, physics_val_csv, path_to_physics_data), title="Tule Daily Dryest Year Timeseries", backend="mpl", metrics=metric_list, start_date=dryest_start_val, end_date=dryest_end_val, metrics_out="tule_daily_val_dry_metrics.csv", ts_out="tule_daily_val_dry_combined_ts.csv", fig_out="tule_daily_val_dry_model_comparison.png", legend_font=20, axis_font=22)
else:
    plot_timeseries_comparison(source=combined_df_val, title="Tule Daily Dryest Year Validation Timeseries", backend="mpl", metrics=metric_list, start_date=dryest_start_val, end_date=dryest_end_val, metrics_out="tule_daily_val_dry_metrics.csv", ts_out="tule_daily_val_dry_combined_ts.csv", fig_out="tule_daily_val_dry_model_comparison.png", legend_font=20, axis_font=22)

In [ ]:
if not skip_best_model_test:
    plot_timeseries_comparison(source=(no_physics_test_csv, physics_test_csv, path_to_physics_data), title="Tule Daily Driest Year Test Timeseries", backend="mpl", metrics=metric_list, start_date=dryest_start_test, end_date=dryest_end_test, metrics_out="tule_daily_test_dry_metrics.csv", ts_out="tule_daily_test_dry_combined_ts.csv", fig_out="tule_daily_test_dry_model_comparison.png", legend_font=20, axis_font=22)
else:
    plot_timeseries_comparison(source=combined_df, title="Tule Daily Driest Year Test Timeseries", backend="mpl", metrics=metric_list, start_date=dryest_start_test, end_date=dryest_end_test, metrics_out="tule_daily_test_dry_metrics.csv", ts_out="tule_daily_test_dry_combined_ts.csv", fig_out="tule_daily_test_dry_model_comparison.png", legend_font=20, axis_font=22)

In [ ]:
if not skip_best_model_test:
    plot_timeseries_comparison(source=(no_physics_test_csv, physics_test_csv, path_to_physics_data), title="Tule Daily Driest Year Test Timeseries – Interactive", backend="plotly", metrics=metric_list, start_date=dryest_start_test, end_date=dryest_end_test, metrics_out="tule_daily_test_dry_metrics.csv", ts_out="tule_daily_test_dry_combined_ts.csv", legend_font=12, axis_font=22)
else:
    plot_timeseries_comparison(source=combined_df, title="Tule Daily Driest Year Test Timeseries – Interactive", backend="plotly", metrics=metric_list, start_date=dryest_start_test, end_date=dryest_end_test, metrics_out="tule_daily_test_dry_metrics.csv", ts_out="tule_daily_test_dry_combined_ts.csv", legend_font=12, axis_font=22)

##### Gridded Timeseries Plots - Validation and Testing

In [ ]:
if not skip_best_model_validation:
    ts_triptych_v3((no_physics_val_csv, physics_val_csv, path_to_physics_data),wet_start=wettest_start_val, wet_end=wettest_end_val, dry_start=dryest_start_val, dry_end=dryest_end_val, save_path="tule_daily_TS_validation.png", legend_font=12, legend_boxpad=0.5, axis_font=12, date_fmt="%d-%b-%Y", figsize=(10, 10), dpi=600, hspace=0.2, main_title="Tule Daily Validation Period Timeseries Across Models", main_title_font=14, main_title_y=0.99, main_title_pad=0.05, row_titles=("Full Validation period","Most-wet water-year","Most-dry water-year"))

else:
    ts_triptych_v3(combined_df_val, wet_start=wettest_start_val, wet_end=wettest_end_val, dry_start=dryest_start_val, dry_end=dryest_end_val, save_path="tule_daily_TS_validation.png", legend_font=12, legend_boxpad=0.5, axis_font=12, date_fmt="%d‑%b‑%Y", figsize=(10, 10), dpi=600, hspace=0.2,main_title = "Tule Daily Validation Timeseries Across Models", main_title_font=14, main_title_y = 0.99, main_title_pad = 0.05, row_titles=("Full validation period", "Most‑wet water‑year", "Most‑dry water‑year"))

In [ ]:
if not skip_best_model_test:
    ts_triptych_v3((no_physics_test_csv, physics_test_csv, path_to_physics_data), wet_start=wettest_start_test, wet_end=wettest_end_test,dry_start=dryest_start_test, dry_end=dryest_end_test, save_path="tule_daily_TS_testing.png", legend_font=12, legend_boxpad=0.5, axis_font=12, date_fmt="%d-%b-%Y", figsize=(10, 10), dpi=600, hspace=0.2, main_title="Tule Daily Test Period Timeseries Across Models", main_title_font=14, main_title_y=0.99, main_title_pad=0.05, row_titles=("Full Testing period","Most-wet water-year","Most-dry water-year"))

else:
    ts_triptych_v3(combined_df, wet_start=wettest_start_test, wet_end=wettest_end_test, dry_start=dryest_start_test, dry_end=dryest_end_test, save_path="tule_daily_TS_testing.png", legend_font=12, legend_boxpad=0.5, axis_font=12, date_fmt="%d‑%b‑%Y", figsize=(10, 10), dpi=600, hspace=0.2, main_title = "Tule Daily Test Period Timeseries Across Models", main_title_font=14, main_title_y = 0.99, main_title_pad = 0.05, row_titles=("Full Testing period", "Most‑wet water‑year", "Most‑dry water‑year"))

##### Gridded Scatter Plots - Testing

In [ ]:
if not skip_best_model_test:
    scatter_triptych_pngs_v3((no_physics_test_csv, physics_test_csv, path_to_physics_data), wet_start=wettest_start_test, wet_end=wettest_end_test, dry_start=dryest_start_test, dry_end=dryest_end_test, out_dir="tule_daily_scatter", layout="horizontal", square_side=4.5, legend_font=16, axis_font=16, point_size=28, top_pad=.90, suptitle_y=1.04, dpi=600, row_titles=("Tule Daily – Full test period", "Tule Daily – Wettest water-year", "Tule Daily – Driest water-year"), resolution="daily")
else:
    scatter_pngs = scatter_triptych_pngs_v3(combined_df, wet_start = wettest_start_test, wet_end = wettest_end_test, dry_start = dryest_start_test,  dry_end = dryest_end_test, out_dir = "tule_daily_scatter", layout = "horizontal", square_side  = 4.5, legend_font  = 16, axis_font = 16, point_size = 28, top_pad = .90, suptitle_y = 1.04, dpi = 600, row_titles = ("Tule Daily – Full test period", "Tule Daily – Wettest water‑year", "Tule Daily – Driest water‑year"), resolution = "daily")